### Caution : Executeing this notebook may take some time because of my self-implemented naive bayes and huge number of features for each observation. So please try to run it on GPU or some powerful CPU .I myself used Floydhub CPU for training it and it took quite sometime !!!!!!!

In [83]:
import numpy as np

* Fetching train and test ___20newsgroups datasets__ from inbuilt datasets of __sklearn__

In [84]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
twenty_test =  fetch_20newsgroups(subset='test',shuffle=True, random_state=42)

In [85]:
twenty_train.target ## Categories for classification

array([7, 4, 4, ..., 3, 1, 8])

* Training data

In [86]:
text= twenty_train.data  ## training documents
types=twenty_train.target ## training labels

* Testing data

In [87]:
text_test = twenty_test.data  # testing documents
types_test = twenty_test.target # testing labels

* Importing stopwords

In [88]:
print(twenty_train.data[4]) ## this is what each text looks like

From: jcm@head-cfa.harvard.edu (Jonathan McDowell)
Subject: Re: Shuttle Launch Question
Organization: Smithsonian Astrophysical Observatory, Cambridge, MA,  USA
Distribution: sci
Lines: 23

From article <C5owCB.n3p@world.std.com>, by tombaker@world.std.com (Tom A Baker):
>>In article <C5JLwx.4H9.1@cs.cmu.edu>, ETRAT@ttacs1.ttu.edu (Pack Rat) writes...
>>>"Clear caution & warning memory.  Verify no unexpected
>>>errors. ...".  I am wondering what an "expected error" might
>>>be.  Sorry if this is a really dumb question, but
> 
> Parity errors in memory or previously known conditions that were waivered.
>    "Yes that is an error, but we already knew about it"
> I'd be curious as to what the real meaning of the quote is.
> 
> tom


My understanding is that the 'expected errors' are basically
known bugs in the warning system software - things are checked
that don't have the right values in yet because they aren't
set till after launch, and suchlike. Rather than fix the code
and possibly i

In [89]:
from sklearn.feature_extraction import text
stopwords=[x for x in set(np.loadtxt('stopwords.txt',dtype= str , delimiter=',').tolist())]
stopwords = list(text.ENGLISH_STOP_WORDS.union(stopwords))
stopwords += ['from','subject','organization','lines']
stopwords = [i for i in set(stopwords)]
len(stopwords)

629

In [90]:
import re 
## i have used this library because it converts a string/document
# into a list of words only (i.e excluding all charaters but not alphanumeric terms or numeric)

* Creating __Vocabulary__ for __extracting__ out __features__ later on as __WordCounts__ for  words not present in __stopwords__ for each document in our __Training Dataset__.

* This __Vocabulary__ will be used __Globally__ i.e for both training and testing. 
We are not going to create a separate vocabulary for test data

In [33]:
def f(t):    ## 't' is list of all documents passed to the function as argument
    word_count={}  ## Dictionary that contains wordcount corresponding to each word in the document
    for i in t:
        for k in [x.lower() for x in re.findall(r'\w+',i.strip())]:# This statement first converts the document that we fetch in every iteration into a list of words only (i.e excluding all charaters but not alphanumeric terms) and then "k" iterates over that list
            if k not in stopwords and len(k)>3 and k.isalpha(): ##condition for the word being checked  does not lie in stopword dataset. 
                if k not in word_count.keys(): # if the above condition is true then start initializing & counting of words in dictionary
                    word_count[k]=1 
                else:
                    word_count[k]+=1
    return word_count
dict_ = f(text) 
# dict_ is our required Vocabulary containing 
# count of each word in all documents but not in stopwords

* Now __1. Sorting the dict_ __
* __2. __Converting __dict___ into a list of tuples in which each tuple contains respective __keys__ and __values__ of __dict___
* The __modified__ vocabulary present in as a list of tuples as mentioned above is named as __vocabulary_dict__.

In [34]:
s = [(k, dict_[k]) for k in sorted(dict_, key=dict_.get, reverse=True)]# returns a list containing the corresponding keys and values of dict_ as individual tuples in the list itself 
vocabulary_dict=[] # for modification of dict_ into a list of tuples
values,keys=[],[] # creating respective lists for respective values and keys in dict_
for i,j in s:
    if j>120: ## condition which decides upto how much frequency the words need to be retained in the final VOCABULARY
        vocabulary_dict.append((i,j)) ## the above statement thus controls feature length
        keys.append(i)
        values.append(j)

In [35]:
s[0] ## showing you individual element of the list of tuples mentioned above

('lines', 11835)

* Total no. of different words whose __count__ is taken as a __feature_value__ using above build vocabulary

In [11]:
# keys and values created in above cell are lists that actually contain 
# corresponding keys and values present in vocabulary_dict
len(keys) # this gives us the feature length or the no. of words used for building features

2074

* Function for calculating __feature_array__ and __modified_feature_indices__ with the help of above build vocabulary

In [12]:
def g(s):    #### s is an array containing documents passed to the function as argument
    feature_array=[] # this is a 2D list(as we'll see at the end of the fn.) in which each element (which is a list) contains count of only those words which are present in vocabulary for each document 
    modified_feature_indices = [] ## this is also 2D list(as we'll see at the end of the fn.) which contains feature/word index [considering the most_common word is our first feature(index = 1)]present in vocabulary which has non-zero value/count in the current document with which the current list of indices is associated
    
    for i in s:
        r=1
        m = []
        individual_count={}
        for h in keys:
            if h not in individual_count.keys(): #initializing dictionary containing word count corresponding to each document (in each iteration) for the words present in vocabulary only
                individual_count[h]=0
        for k in [x.lower() for x in re.findall(r'\w+',i.strip())]: # This statement first converts the document that we fetch in every iteration into a list of words only(excluding all characters) and then "k" iterates over that list
            if k in keys:
                if k in individual_count.keys():
                    individual_count[k]+=1 #it increments the count for each word identified in vocabulary
        for x in individual_count.values():
            if x!=0:         # checking condition for whether the word has non-zero count.
                m.append(r)
            r+=1 
        modified_feature_indices.append(m)
        feature_array.append([x for x in individual_count.values()])# appending these individual list of word counts into feature_array 
    return feature_array,modified_feature_indices
feature_array,modified_feature_indices_for_train=g(text) ## training features and list of indices for non-zero count of words in each document
feature_array_test,modified_feature_indices_for_test= g(text_test) # testing features and list of indices for non-zero count of words in each document

## Now training the features on different models
* First Using Sklearn

In [91]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
clf = MultinomialNB()

In [10]:
clf.fit(feature_array,types) ## fitting the training data in classifier

* printing __accuracy_scores__ and __classification reports__ for training and testing datasets

__For Testing Dataset :__

In [15]:
print(classification_report(types_test,clf.predict(feature_array_test)))
print(accuracy_score(types_test,clf.predict(feature_array_test)))

             precision    recall  f1-score   support

          0       0.63      0.66      0.65       319
          1       0.57      0.63      0.60       389
          2       0.55      0.68      0.61       394
          3       0.59      0.56      0.58       392
          4       0.62      0.60      0.61       385
          5       0.86      0.57      0.68       395
          6       0.65      0.86      0.74       390
          7       0.73      0.78      0.75       396
          8       0.71      0.86      0.78       398
          9       0.82      0.83      0.83       397
         10       0.90      0.88      0.89       399
         11       0.89      0.82      0.85       396
         12       0.60      0.58      0.59       393
         13       0.80      0.68      0.73       396
         14       0.86      0.80      0.83       394
         15       0.83      0.82      0.82       398
         16       0.67      0.81      0.73       364
         17       0.95      0.78      0.86   

# Now using self-implemented naive_bayes:

In [92]:
def fit(X_train, Y_train):         ## it returns the dictionary corresponding to the word count for different classes 
    result = {}
    class_values = set(Y_train)
    for current_class in class_values:
        result[current_class] = {}
        result["total_data"] = len(Y_train)
        current_class_rows = (Y_train == current_class)
        X_train_current = X_train[current_class_rows]
        Y_train_current = Y_train[current_class_rows]
        num_features = X_train.shape[1]
        result[current_class]["total_count"] = len(Y_train_current)
        for j in range(1, num_features + 1):
            result[current_class][j] = (X_train_current[:, j - 1]).sum()
            
    return result

In [93]:
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[current_class].keys()) - 1
    p=0
    for j in range(1, x[-1]+ 1):   
        if x[p] != j: ## Condition that lets us to proceed to calculate probability for only those words that are actually present in that documrnt/email
            continue
        else:
            count_current_class_with_value_j = dictionary[current_class][j] + 1
            count_current_class = (sum(dictionary[current_class].values())-dictionary[current_class]["total_count"]) + num_features  ## for finding the total wordcount for a specific word that we looking at in this particular iteration for each class
            current_j_probablity = np.log(count_current_class_with_value_j) - np.log(count_current_class)
            output = output + current_j_probablity
            p+=1

    return output

In [94]:
def predictSinglePoint(dictionary, x): # same as discussed in class without any change
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        p_current_class = probability(dictionary, x, current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
        
    return best_class

In [95]:
def predict(dictionary, X_test):    # here x_test is 'modified_feature_indices_for_test'
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
        
    return y_pred

* Convering our __training__ and __testing__ data into __numpy_arrays__ as our self-implemented code works only numpy_arrays.

__Training dataset __ 

In [20]:
feature_array = np.array(feature_array)
modified_feature_indices_for_train = np.array(modified_feature_indices_for_train)
types = np.array(types)

__Testing dataset__

In [21]:
modified_feature_indices_for_test = np.array(modified_feature_indices_for_test)
types_test = np.array(types_test)

In [22]:
dictionary = fit(feature_array,types) ## The model has been fitted now 

* __PREDICTIONS__

In [24]:
types_test_predicted = predict(dictionary,modified_feature_indices_for_test)

* Printing __Classification Reports__  for testing datasets

In [26]:
## Testing dataset
print(classification_report(types_test,types_test_predicted))
print(accuracy_score(types_test,types_test_predicted))

             precision    recall  f1-score   support

          0       0.62      0.70      0.66       319
          1       0.54      0.65      0.59       389
          2       0.54      0.62      0.58       394
          3       0.63      0.53      0.57       392
          4       0.60      0.62      0.61       385
          5       0.86      0.55      0.67       395
          6       0.67      0.86      0.75       390
          7       0.68      0.78      0.72       396
          8       0.68      0.86      0.76       398
          9       0.84      0.85      0.84       397
         10       0.94      0.87      0.90       399
         11       0.89      0.81      0.85       396
         12       0.58      0.58      0.58       393
         13       0.79      0.70      0.74       396
         14       0.87      0.77      0.82       394
         15       0.86      0.85      0.85       398
         16       0.67      0.81      0.73       364
         17       0.97      0.73      0.83   

* Now I am using __nltk__ for __feature extraction__ and then for testing using __nltk.NaiveBayesClassifier,SKlearn MultinomialNB__ and my __self-implemented__ naive bayes:

In [96]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [97]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [98]:
text= twenty_train.data  ## training documents
types=twenty_train.target ## training labels

text_test = twenty_test.data  # testing documents
types_test = twenty_test.target # testing labels

* __Lemmatization__

In [99]:
def reduced_words(doc):
    all_words = []
    documents = []
    res_word_list = []
    for x in doc:
        l = [i for i in re.findall(r'\w+',x)]
        new_list = []
        pos = pos_tag(l)
        for i,j in pos:
            if i.lower() not in stopwords and i.isalpha() and len(i)>2:
                if j.startswith('J'):
                    new_list.append(lemmatizer.lemmatize(i,pos = wordnet.ADJ ))
                elif j.startswith('V'):
                    new_list.append(lemmatizer.lemmatize(i,pos = wordnet.VERB ))
                elif j.startswith('R'):
                    new_list.append(lemmatizer.lemmatize(i,pos = wordnet.ADV ))
                elif j.startswith('N'):
                    new_list.append(lemmatizer.lemmatize(i,pos = wordnet.NOUN ))
                else:
                    new_list.append(i)
        for word in new_list:
            all_words.append(word)
        res_word_list.append(new_list)
        documents.append(' '.join(new_list))
    return  documents,all_words,res_word_list
documents_train,all_words,res_word_list_train = reduced_words(text)
documents_test,ignore,res_word_list_test = reduced_words(text_test)

* __NOTE :__ Play around __min_df__ and __max_df__ in __Count_vec__ & __Tfidf_vec__ to control the number of most common words considered for make features.

In [100]:
documents_train[0]

'lerxst wam umd thing car Nntp Posting Host wam umd University Maryland College Park wonder enlighten car day door sport car look late early call Bricklin door small addition bumper separate rest body model engine specs year production car history info funky car mail bring neighborhood Lerxst'

In [101]:
all_words[-5:-1]

['tomorrow', 'Guess', 'miss', 'baby']

In [102]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
count_vec = CountVectorizer(ngram_range = (1,2),min_df = 0.005,max_df = 0.95,analyzer = 'word')
tfidf_vec = TfidfVectorizer(ngram_range = (1,2),min_df = 0.005,max_df = 0.95,analyzer = 'word')

In [103]:
x_train_count = count_vec.fit_transform(documents_train)
x_test_count  = count_vec.transform(documents_test)
x_train_count.shape,x_test_count.shape

((11314, 3186), (7532, 3186))

In [104]:
x_train_tfidf = tfidf_vec.fit_transform(documents_train)
x_test_tfidf  = tfidf_vec.transform(documents_test)
x_train_tfidf.shape,x_test_tfidf.shape

((11314, 3186), (7532, 3186))

In [105]:
x_train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

# First using Inbuilt MultinomialNB()

* __Using WordCounts__

In [68]:
cmb = MultinomialNB()
cmb.fit(x_train_count,types) ## Using Word-Counts

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [69]:
print(classification_report(types_test,cmb.predict(x_test_count)))
print(accuracy_score(types_test,cmb.predict(x_test_count)))

             precision    recall  f1-score   support

          0       0.74      0.73      0.74       319
          1       0.58      0.72      0.64       389
          2       0.75      0.62      0.68       394
          3       0.63      0.63      0.63       392
          4       0.66      0.77      0.71       385
          5       0.87      0.74      0.80       395
          6       0.69      0.81      0.74       390
          7       0.81      0.84      0.82       396
          8       0.82      0.91      0.86       398
          9       0.90      0.89      0.89       397
         10       0.92      0.94      0.93       399
         11       0.92      0.87      0.89       396
         12       0.71      0.64      0.67       393
         13       0.89      0.73      0.80       396
         14       0.86      0.88      0.87       394
         15       0.88      0.90      0.89       398
         16       0.74      0.87      0.80       364
         17       0.97      0.85      0.91   

* __Using Tfidf__

In [70]:
tmb = MultinomialNB()
tmb.fit(x_train_tfidf,types) ## usinf TF-IDF

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [71]:
print(classification_report(types_test,tmb.predict(x_test_tfidf)))
print(accuracy_score(types_test,tmb.predict(x_test_tfidf)))

             precision    recall  f1-score   support

          0       0.77      0.67      0.72       319
          1       0.64      0.69      0.66       389
          2       0.69      0.70      0.70       394
          3       0.63      0.69      0.66       392
          4       0.78      0.73      0.75       385
          5       0.81      0.79      0.80       395
          6       0.76      0.78      0.77       390
          7       0.83      0.84      0.83       396
          8       0.84      0.90      0.87       398
          9       0.88      0.90      0.89       397
         10       0.86      0.95      0.91       399
         11       0.89      0.92      0.90       396
         12       0.73      0.61      0.67       393
         13       0.88      0.74      0.80       396
         14       0.82      0.91      0.86       394
         15       0.68      0.95      0.79       398
         16       0.71      0.90      0.79       364
         17       0.93      0.90      0.91   

# Using Self-Implemented Naive_Bayes :

In [106]:
def get_features_for_self_implementation(array):    #### s is an array obtained from count_vect/tfidf_vect
    modified_feature_indices = [] ## this is also 2D list(as we'll see at the end of the fn.) which contains feature/word index [considering the most_common word is our first feature(index = 1)]present in vocabulary which has non-zero value/count in the current document with which the current list of indices is associated
    
    for feature in array:
        m = []
        for j in range(len(feature)):
            if feature[j]!=0:
                m.append(j)
        modified_feature_indices.append(m)
    return modified_feature_indices

modified_feature_for_test_count = np.array(get_features_for_self_implementation(np.array(x_test_count.todense()))) ## Testing list of indices for non-zero count of words in each document
modified_feature_for_test_tfidf = np.array(get_features_for_self_implementation(np.array(x_test_tfidf.todense()))) ## Testing list of indices for non-zero TF-IDFs of words in each document

In [107]:
np.array(x_train_count.todense()).shape,types.shape ## checking the  shapes of our models

((11314, 3186), (11314,))

In [108]:
dictionary_count = fit(np.array(x_train_count.todense()),types) ##calling fit function to create a dictionary for WordCount model
dictionary_tfidf = fit(np.array(x_train_tfidf.todense()),types) ##calling fit function to create a dictionary for TF-IDF model

* __PREDICTIONS__

In [110]:
types_test_predicted_count = predict(dictionary_count,modified_feature_for_test_count) ## getting predictions for WordCount model

In [111]:
types_test_predicted_tfidf = predict(dictionary_tfidf,modified_feature_for_test_tfidf) ## getting predictions for TF-IDF model

* __Accuracy_score/Classification Report__ for __Word-Count__ model

In [112]:
print(classification_report(types_test,types_test_predicted_count))
print(accuracy_score(types_test,types_test_predicted_count))

             precision    recall  f1-score   support

          0       0.25      0.24      0.25       319
          1       0.17      0.11      0.13       389
          2       0.04      0.00      0.00       394
          3       0.13      0.04      0.06       392
          4       0.11      0.09      0.10       385
          5       0.58      0.07      0.13       395
          6       0.05      0.14      0.08       390
          7       0.14      0.17      0.15       396
          8       0.11      0.29      0.15       398
          9       0.24      0.14      0.17       397
         10       0.06      0.02      0.02       399
         11       0.30      0.22      0.25       396
         12       0.10      0.08      0.09       393
         13       0.11      0.22      0.15       396
         14       0.19      0.23      0.21       394
         15       0.47      0.15      0.22       398
         16       0.16      0.16      0.16       364
         17       0.62      0.23      0.34   

* __Accuracy_score/Classification Report__ for __TF-IDF__ model

In [113]:
print(classification_report(types_test,types_test_predicted_tfidf))
print(accuracy_score(types_test,types_test_predicted_tfidf))

             precision    recall  f1-score   support

          0       0.25      0.30      0.27       319
          1       0.19      0.23      0.21       389
          2       0.08      0.03      0.04       394
          3       0.11      0.09      0.10       392
          4       0.13      0.12      0.13       385
          5       0.30      0.16      0.21       395
          6       0.08      0.09      0.09       390
          7       0.13      0.11      0.12       396
          8       0.16      0.29      0.21       398
          9       0.28      0.23      0.25       397
         10       0.20      0.10      0.14       399
         11       0.33      0.25      0.28       396
         12       0.13      0.09      0.11       393
         13       0.17      0.12      0.14       396
         14       0.31      0.32      0.32       394
         15       0.48      0.24      0.32       398
         16       0.14      0.07      0.09       364
         17       0.54      0.32      0.40   

* Now preparing __feature_set__ format required for __nltk classifiers__:

In [38]:
feat = nltk.FreqDist(all_words)
vocabulary = [i for i,j in feat.most_common(3000)] ## our vocabulary for features

In [39]:
def get_features(res_word_list,labels):
    feature_set = []
    for i in range(len(res_word_list)):
        dic = {}
        for j in vocabulary:
            dic[j] = (j in res_word_list[i])
        feature_set.append((dic,labels[i]))
    return feature_set
feature_set_train = get_features(res_word_list_train,types) ## Getting training features in form of 
feature_set_test = get_features(res_word_list_test,types_test)

In [40]:
from nltk import NaiveBayesClassifier
nbc = NaiveBayesClassifier.train(feature_set_train)

In [41]:
nltk.classify.accuracy(nbc,feature_set_test)

0.6802973977695167

# Final Conclusion [Comparison]:
  * Highest Accuracy Using __SklearnMultinomialNB[By using self-implemented features]__ = 0.7158789166224111
  * Highest Accuracy Using __SklearnMultinomialNB[By using Features with the help of CountVectorizer]__ = 0.7810674455655868
  * Highest Accuracy Using __SklearnMultinomialNB[By using Features with the help of TfidfVectorizer]__ = 0.7843866171003717

  * Highest Accuracy Using __Self-Implemented Naive_Bayes[By using self-implemented features]__ = 0.7118959107806692
  * Highest Accuracy Using __Self-Implemented Naive_Bayes[By using Features with the help of CountVectorizer]__ = 
  * Highest Accuracy Using __Self-Implemented Naive_Bayes[By using Features with the help of TfidfVectorizer]__ = 

* Highest Accuracy Using __nltk NaiveBayesClassifier__ = 0.6802973977695167

 * As we can see there is only a slight difference in accuracy_score b/w two models.This difference can be accounted by the fact that MultinomialNB contains three other parameters to tune __(alpha, fit_prior,   class_prior)__ while our self-implemented Naive_Bayes  is only a simple application of  bayes_probability theory with no other parameters to care for optimisation.